<a href="https://colab.research.google.com/github/86HenriqueSilva/Treinamento_de_ia-s/blob/main/IA_PyTorch_REDE_NEURAL_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

IA_PyTorch_REDE_NEURAL_02_

import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# Montar o Google Drive para acessar o arquivo de dados
from google.colab import drive
drive.mount('/content/drive')

# URL do arquivo CSV no Google Drive contendo os dados do histórico da Mega Sena
url = "https://drive.google.com/uc?id=1BXzIxSNSK03Q73OfvC_CCDoJSOZ_LI2n"

# Carregar os dados do arquivo CSV para uma estrutura de dados tabular
print("Carregando os dados do histórico da Mega Sena...")
dados = pd.read_csv(url)

# Remover colunas de data
dados = dados.drop(columns=['Data'])

# Definir janela de histórico (quantidade de concursos anteriores)
janela = 10

# Função para criar janelas de histórico dos dados
def criar_janelas(dados, janela):
    """
    Função para criar janelas de histórico dos dados.

    Args:
        dados (DataFrame): DataFrame contendo os dados do histórico da Mega Sena.
        janela (int): Tamanho da janela de histórico.

    Returns:
        X (array): Array contendo as janelas de histórico.
        y (array): Array contendo os dados do próximo concurso após cada janela.
    """
    X, y = [], []
    for i in range(len(dados)-janela-1):
        janela_dados = dados.iloc[i:(i+janela), 1:].values.flatten()
        X.append(janela_dados)
        proximo_concurso = dados.iloc[i+janela, 1:].values
        y.append(proximo_concurso)
    return np.array(X), np.array(y)

# Criar janelas de histórico dos dados
X, y = criar_janelas(dados, janela)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definir modelo da rede neural LSTM
class MegaSenaPredictorLSTM(nn.Module):
    """
    Classe para o modelo da rede neural LSTM para previsão da Mega Sena.

    Args:
        input_size (int): Tamanho da entrada.
        hidden_size (int): Tamanho da camada oculta.
        output_size (int): Tamanho da saída.
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(MegaSenaPredictorLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Definir hiperparâmetros da rede neural LSTM
input_size_lstm = janela * 6  # 6 bolas em cada concurso
hidden_size_lstm = 512
output_size_lstm = 6

# Instanciar o modelo da rede neural LSTM
modelo_lstm = MegaSenaPredictorLSTM(input_size_lstm, hidden_size_lstm, output_size_lstm)

# Definir função de perda e otimizador
criterion_lstm = nn.MSELoss()
optimizer_lstm = optim.Adam(modelo_lstm.parameters(), lr=0.001)

# Definir número de épocas e tamanho do batch
num_epochs_lstm = 100
batch_size_lstm = 512

# Função para pós-processamento das previsões para evitar repetições de números
def processar_previsoes(previsoes):
    """
    Função para processar as previsões e evitar repetições de números.

    Args:
        previsoes (numpy.ndarray): Array contendo as previsões do modelo.

    Returns:
        previsoes_processadas (numpy.ndarray): Array contendo as previsões processadas.
    """
    previsoes_processadas = []
    for numeros in previsoes:
        numeros_unicos = set()
        numeros_processados = []
        for numero in numeros:
            while numero in numeros_unicos:
                numero = np.random.randint(61)  # Gerar um novo número único
            numeros_processados.append(numero)
            numeros_unicos.add(numero)
        previsoes_processadas.append(numeros_processados)
    return np.array(previsoes_processadas)

# Treinar a rede neural LSTM
print("Treinando a rede neural LSTM...")
for epoch in tqdm(range(num_epochs_lstm)):
    # Embaralhar os dados de treinamento
    indices = np.random.permutation(len(X_train))
    X_train_shuffled = X_train[indices]
    y_train_shuffled = y_train[indices]

    # Dividir os dados em batches
    for i in range(0, len(X_train), batch_size_lstm):
        inputs = torch.tensor(X_train_shuffled[i:i+batch_size_lstm], dtype=torch.float32).unsqueeze(1)
        targets = torch.tensor(y_train_shuffled[i:i+batch_size_lstm], dtype=torch.float32)
        optimizer_lstm.zero_grad()
        outputs = modelo_lstm(inputs)
        loss = criterion_lstm(outputs, targets)
        loss.backward()
        optimizer_lstm.step()

print("Treinamento concluído.")

# Avaliar o modelo LSTM
modelo_lstm.eval()
with torch.no_grad():
    inputs_lstm = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
    outputs_lstm = modelo_lstm(inputs_lstm).numpy()

# Processar as previsões para evitar repetições de números
previsoes_inteiras_lstm = np.round(outputs_lstm).astype(int)
previsoes_processadas_lstm = processar_previsoes(previsoes_inteiras_lstm)

# Verificar se as previsões processadas estão vazias
print("Número de previsões processadas:", len(previsoes_processadas_lstm))

# Verificar se há algum problema antes de exibir as previsões processadas
print("Previsões processadas para o próximo sorteio da Mega Sena:")
print(previsoes_processadas_lstm)